# CS 328 Introduction to Data Science - Homework 1

Author: Nupoor Assudani

Roll number: 23110224

Collaborators: Aryan Solanki 23110049

In [2]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import struct
from array import array
%matplotlib inline
import random
from sklearn.metrics import rand_score
from scipy.spatial.distance import pdist, squareform
import time
import heapq

## Question 1

Suppose you define a clustering objective in the following manner – give a partitioning $\mathbb{C} = \{C_1, . . . C_k\}$,
define $$\text{cost}(\mathbb{C}) = \sum_{i} \frac{1}{|C_i|} \sum_{x,y \in C_i} \|x - y\|_2^2$$

i.e. cost of a cluster is the sum of all pairwise squared distances. Give an algorithm for this.


### Answer

In [19]:
def calc_obj(X, centroids):  
    x_reshaped = X[:, np.newaxis] # reshaping for the subtraction
    distances = np.linalg.norm(x_reshaped - centroids, axis=2)  
    labels = np.argmin(distances, axis=1)  # choosing the centroid each point is closest to as its cluster centre
    obj = 0 
    clusters = {i: [] for i in range(len(centroids))}  
    for i, label in enumerate(labels):
        clusters[label].append(X[i])
    for cluster in clusters.values(): # iterating through each cluster
        if len(cluster) > 1:
            cl = np.array(cluster)
            cl_reshaped = cl[:, np.newaxis] # reshaping for the subtraction
            sum_sqrd_l2_norms = np.sum((cl_reshaped - cl) ** 2, axis=(1, 2)) # for each
            obj += np.sum(sum_sqrd_l2_norms) / len(cluster)
    return obj

In [20]:
X_test = np.array([[1, 2], [2, 3], [3, 4], [4, 5], [5, 6]])
centroids_test = np.array([[2, 3], [4.5, 5.5]])
print(calc_obj(X_test, centroids_test))

10.0


## Question 2

For the $k$-means problem, show that there is at most a factor of four ratio between the optimal value when
we either require all cluster centers to be data points or allow arbitrary points to be centers.

### Answer

The optimal value when we allow cluster centres to be arbitrary points $$\text{cost}(\mathbb{C}) = \sum_{i=1}^{k} \sum_{x \in C_i} \|x - \mu_i\|_2^2$$ where $\mu_i$ is the centroid of $C_i$ or the mean of all points in $C_i$ and the clustering $\mathbb{C} = \{C_1, . . . C_k\}$.

The optimal value when we require cluster centres to be data points $$\text{cost}(\mathbb{C}_p) = \sum_{i=1}^{k} \sum_{x \in C_{p_i}} \|x - \mu_{p_i}\|_2^2$$ where $\mu_{p_i}$ is the centroid of $C_{p_i}$ or the data point closest to the mean of all points in $C_{p_i}$ and the clustering $\mathbb{C}_p = \{C_{p_1}, . . . C_{p_k}\}$.

Here we assume that both the partitionings are the same.

We need to prove that $\text{cost}(\mathbb{C}_p) \leq 4 \text{ cost}(\mathbb{C})$.

$\text{Proof}$

Using the triangle inequality, $$\|x - \mu_{p_i}\|_2 \leq \|x - \mu_i\|_2 + \|\mu_i - \mu_{p_i}\|_2$$

Squaring on both sides, $$\|x - \mu_{p_i}\|_2^2 \leq \|x - \mu_i\|_2^2 + \|\mu_i - \mu_{p_i}\|_2^2 + 2 \times \|x - \mu_i\|_2 \times \|\mu_i - \mu_{p_i}\|_2$$

$\mu_{p_i}$ is the point closest to $\mu_i$ therefore for any value of $x$, $\|\mu_i - x\|_2 \geq \|\mu_{p_i} - \mu_i\|_2$.

$$\therefore \|x - \mu_{p_i}\|_2^2 \leq \|x - \mu_i\|_2^2 + \|\mu_i - x\|_2^2 + 2 \times \|x - \mu_i\|_2 \times \|\mu_i - x\|_2$$

$$\therefore \|x - \mu_{p_i}\|_2^2 \leq 4 \|x - \mu_i\|_2^2$$

$$\therefore \sum_{i=1}^{k} \sum_{x \in C_{p_i}}\|x - \mu_{p_i}\|_2^2 \leq \sum_{i=1}^{k} \sum_{x \in C_i}4 \|x - \mu_i\|_2^2$$

$$\text{cost}(\mathbb{C}_p) \leq 4\text{ cost}(\mathbb{C})$$

This proves that for the $k$-means problem, there is at most a factor of four ratio between the optimal value when we either require all cluster centers to be data points or allow arbitrary points to be centers. $\quad \blacksquare$

## Question 3

Create a random variable $X$ for which Markov’s inequality is tight. Give proof for your answer. If it is tight, then why are we using other inequalities e.g. Chebyshev and Chernoff?

### Answer

Markov's inequality states that $ P(X \geq t) \leq \frac{E[X]}{t}, \quad \text{for } X \geq 0 \text{ and } t > 0. $

It is called tight when $ P(X \geq t) = \frac{E[X]}{t}$

Let

$$
P(X = x) =
\begin{cases} 
\frac{1}{2}, & \text{if } x = 0 \\ 
\frac{1}{2}, & \text{if } x = 1
\end{cases}
$$

This is a Bernoulli distribution with parameter \( p = 0.5 \):

This can be written as $X \sim \text{Bernoulli}(p), \quad \text{where } p = 0.5$

The expectation of X,
$$
\mathbb{E}[X] = \frac{1}{2} \times 0 + \frac{1}{2} \times 1 = \frac{1}{2}
$$

When $t=1$, Markov's inequality is tight in this case
$$P(X=1) = \frac{E[X]}{1} = \frac{0.5}{1} = \frac{1}{2}$$


Even though Markov’s inequality can be tight, it is often loose for other distributions. This is because it only uses the expectation and ignores higher moments like variance.

The advantage of Chebyshev's inequality when compared to Markov's are that it uses both the mean and variance giving a tighter bound for lower values of variance.

The advantage of Chernoff's bound when compared to Markov's inequality is that it uses moment generating functions (i.e. function which gives all moments of the random variable), helping in measuring how "spread out" a random variable is. This allows us to get much sharper estimates on the probability of extreme events. It decreses exponentially, meaning the probability of a large deviation drops much faster.

Thus we prefer bounds other than Markov for stronger results.

## Question 4

Download the MNIST dataset from https://www.kaggle.com/datasets/hojjatk/mnist-dataset/data. We will use the test dataset
and test labels only.

In [3]:
#
# This is from a sample Notebook that demonstrates how to read "MNIST Dataset" from the site https://www.kaggle.com/code/hojjatk/read-mnist-dataset
# The author of the webpage is Hojjat Khodabakhsh
#
# MNIST Data Loader Class
#
class MnistDataloader(object):
    def __init__(self, train_images_filepath, train_labels_filepath, test_images_filepath, test_labels_filepath):
        self.train_images_filepath = train_images_filepath
        self.train_labels_filepath = train_labels_filepath
        self.test_images_filepath = test_images_filepath
        self.test_labels_filepath = test_labels_filepath

    def read_images_labels(self, images_filepath, labels_filepath):
        labels = []
        with open(labels_filepath, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
            labels = array("B", file.read())

        with open(images_filepath, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
            image_data = array("B", file.read())
        images = []
        for i in range(size):
            images.append([0] * rows * cols)
        for i in range(size):
            img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
            img = img.reshape(28, 28)
            images[i][:] = img

        return images, labels

    def load_data(self):
        x_train, y_train = self.read_images_labels(self.train_images_filepath, self.train_labels_filepath)
        x_test, y_test = self.read_images_labels(self.test_images_filepath, self.test_labels_filepath)
        return (x_train, y_train),(x_test, y_test)

In [4]:
train_images_filepath = r"C:\Users\nupoo\Downloads\archive\train-images.idx3-ubyte"
train_labels_filepath = r"C:\Users\nupoo\Downloads\archive\train-labels.idx1-ubyte"
test_images_filepath = r"C:\Users\nupoo\Downloads\archive\t10k-images.idx3-ubyte"
test_labels_filepath = r"C:\Users\nupoo\Downloads\archive\t10k-labels.idx1-ubyte"

mnist_dataloader = MnistDataloader(train_images_filepath, train_labels_filepath, test_images_filepath, test_labels_filepath)
(x_train, y_train), (x_test, y_test) = mnist_dataloader.load_data()

(a) Cluster them first using k-means clustering, $k$ = 10, with kmeans + + initialization (implement the
complete Lloyd’s algorithm yourself). Check the Rand-index of the clustering against the true labels.
Use the sklearn module for rand-index.

In [6]:
def kpp_init(X, k): # X (N, d), k int
    c0 = X[np.random.choice(range(len(X)))]
    centroids = [c0]
    while len(centroids) < k:
        x_reshaped = X[:, np.newaxis] # to (N, 1, D)
        c_arr = np.array(centroids)
        dist_mat = np.linalg.norm(x_reshaped - c_arr, axis = 2) # calculating euclidean distance, (N, k)
        distances = np.min(dist_mat, axis=1) # array of distances from closest centre
        dist_sqrd = distances**2 
        dist_sum = np.sum(dist_sqrd)
        probs = dist_sqrd/dist_sum # D(x_i)^2 / sum(D(x_i)^2)
        new_c = X[np.random.choice(len(X), p=probs)] # chosen randomly with calculated probability
        centroids.append(new_c) # new centroid added to list
    return np.array(centroids)

In [7]:
def kmeans(X, k, max_iters = 100, stop_change = 1e-4, random_state = 42):
    np.random.seed(random_state)
    centroids = kpp_init(X, k)
    for i in range(max_iters):
        x_reshaped = X[:, np.newaxis] # to (N, 1, D)
        distances = np.linalg.norm(x_reshaped - centroids, axis = 2) # calculating euclidean distance, (N, k)
        labels = np.argmin(distances, axis = 1) # assigned centroid that it is closest to (n,)
        new_centroids = np.array([X[np.where(labels == i)].mean(axis=0) if np.any(labels == i) else centroids[i] for i in range(k)]) # current cluster means
        if np.linalg.norm(new_centroids - centroids) < stop_change:
            break
        centroids = new_centroids
    return labels, centroids

In [9]:
x_test = np.array(x_test)
mnist_test = x_test.reshape(10000, 28 * 28)
k = 10  # number of clusters 0-9
labels, centroids = kmeans(mnist_test, k)

In [10]:
ri = rand_score(y_test, labels)
print(f"Rand index on performing k-means clustering with k-means ++ initialization: {ri}")

Rand index on performing k-means clustering with k-means ++ initialization: 0.8922220022002201


(b) Do the same for k-center clustering, $k$ = 10. Implement the greedy algorithm discussed in class.
Report the Rand-index here too.

In [8]:
def k_centre_greedy(X, k):
    c0 = X[np.random.choice(range(len(X)))]
    centroids = [c0]
    for i in range(k-1):
        x_reshaped = X[:, np.newaxis] # to (N, 1, D)
        c_arr = np.array(centroids)
        dist_mat = np.linalg.norm(x_reshaped - c_arr, axis = 2) # calculating euclidean distance, (N, k)
        dist_from_c = np.min(dist_mat, axis = 1) # distances to closest centroid (n,)
        new_c = X[np.argmax(dist_from_c)] # farthest point from the closest existing center
        centroids.append(new_c)
    x_reshaped = X[:, np.newaxis] # to (N, 1, D)
    distances = np.linalg.norm(x_reshaped - np.array(centroids), axis = 2) # calculating euclidean distance, (N, k)
    labels = np.argmin(distances, axis = 1) # assigned centroid that it is closest to (n,)
    return labels, np.array(centroids)

In [12]:
x_test = np.array(x_test)
mnist_test = x_test.reshape(10000, 28 * 28)
k = 10  # number of clusters 0-9
labels, centroids = k_centre_greedy(mnist_test, k)

In [13]:
ri = rand_score(y_test, labels)
print(f"Rand index on performing k-centres clustering with greedy approach: {ri}")

Rand index on performing k-centres clustering with greedy approach: 0.5249141314131414


(c) Run the single linkage agglomeration till there are $k$ = 10 clusters. Report Rand-index here too.

In [21]:
def single_linkage_agglomeration(X, k):
    clusters = {i:[i] for i in range(len(X))} # initially each point is in its own cluster
    which_cluster = {i:i for i in range(len(X))}
    n = len(X)
    time1 = time.time()
    pairwise_distances = squareform(pdist(X, metric='euclidean')) # precomputing distances for efficiency
    heap = [(pairwise_distances[i, j], i, j) for i in range(n) for j in range(i+1, n)] # making a list of pairwise distances
    heapq.heapify(heap) # min-heap created in place
    # time2 = time.time()
    # timer = time2 - time1
    # print(f"Precomputed pairwise distances for efficiency. Time taken = {timer}")
    # t = 0
    while len(clusters)>k:
        t1 = time.time()
        while heap:
            dist, i, j = heapq.heappop(heap) # the closest pair
            c1, c2 = which_cluster[i], which_cluster[j]
            if which_cluster[i] != which_cluster[j]: # if they don't belong to the same cluster, merge their clusters
                break
        clusters[c1].extend(clusters[c2]) # merging two clusters
        for i in clusters[c2]:
            which_cluster[i] = c1
        del clusters[c2]
        # t2 = time.time()
        # t = t2 - t1
        # print(f"Current number of clusters is {len(clusters)}. Time taken for this iteration is {t}.")
    labels = np.zeros(len(X), dtype=int)
    for cluster in list(clusters.keys()): # final labelling
        for i in clusters[cluster]:
            labels[i] = cluster    
    return labels

In [22]:
x_test = np.array(x_test)
mnist_test = x_test.reshape(10000, 28 * 28)
k = 10  # number of clusters 0-9
labels = single_linkage_agglomeration(mnist_test, k)

In [23]:
ri = rand_score(y_test, labels)
print(f"Rand index on performing single-linkage agglomeration: {ri}")

Rand index on performing single-linkage agglomeration: 0.1017039703970397


(d) Run the same algorithms (k-means and k-center) but on a rank-k approximation of the training data
matrix. Note that if $A$ is the training data matrix (images × pixels), then you can just use $U_k Σ_k$ for
the clustering, no need to use $V_k$. Evaluate for $k$ = 2, 5, 10 and report the rand-index values.

In [9]:
k = [2, 5, 10]
for k_val in k:
    U_k, S_k, V_k = np.linalg.svd(x_train)
    A_k = U_k @ np.diag(S_k)
    labels1, centroids = kmeans(A_k, k_val)
    ri1 = rand_score(y_train, labels1)
    print(f"Rand index on performing k-means clustering with k-means ++ initialization after svd with rank {k}: {ri1}")
    labels2, centroids = k_centre_greedy(A_k, k_val)
    ri2 = rand_score(y_train, labels2)
    print(f"Rand index on performing k-centres clustering with greedy approach after svd with rank {k}: {ri2}")

Rand index on performing k-means clustering with k-means ++ initialization after svd with rank [2, 5, 10]: 0.5133793368778369
Rand index on performing k-centres clustering with greedy approach after svd with rank [2, 5, 10]: 0.5119447063006606
Rand index on performing k-means clustering with k-means ++ initialization after svd with rank [2, 5, 10]: 0.7538383450835292
Rand index on performing k-centres clustering with greedy approach after svd with rank [2, 5, 10]: 0.7375379817441402
Rand index on performing k-means clustering with k-means ++ initialization after svd with rank [2, 5, 10]: 0.8293026400440008
Rand index on performing k-centres clustering with greedy approach after svd with rank [2, 5, 10]: 0.7827990055389812


## Question 5

Suppose you have a population of 1 million people, out of which at least 1% are coffee drinkers. You want
to get the estimate of this fraction by using sampling. Give the algorithm and the estimate. What kind
of error bounds can you give with probability 99%?

### Answer

The way to get the estimate for this is randomly sampling n people from the population.

The fraction of people who drink coffee can be defined as the sum of a number of Bernoulli random variables. Each random variable is a Bernoulli random variable, having the value 1 if the person drinks coffee and the value 0 if they don't.

$S_n = X_1 + X_2 + ... + X_n$, where $X_i$ is the probability that the $i^{th}$ person we sample drinks coffee and we sample a total of n people.

We know that at least 1% of the population drinks coffee, therefore $E[X_i] \geq 0.01$ and $E[S_n] \geq n\times 0.01$.

From the Chernoff bound, we know that
$$P(S_n - E[S_n]\geq t)\leq e^{\frac {-2t^2} {n}}$$
$$P(|S_n - E[S_n]|\geq t)\leq 2e^{\frac {-2t^2} {n}}$$

We know that $E[X_i] = \frac {1} {n} E[S_n]$.

We want the probability of the error being within t to be 99%, therefore $$P(|S_n - E[S_n]|\leq t) \geq 0.99$$
$$1 - P(|S_n - E[S_n]|\leq t) \leq 1 - 0.99$$
$$P(|S_n - E[S_n]|\geq t) \leq 0.01$$

Therefore, $$0.01 \geq 2e^{\frac {-2t^2} {n}}$$

If t is the error in the calculation of $S_n$, and $\epsilon$ is the error in the calculation of $X_i$, then we can also write $$0.01 \geq 2e^{-2n\epsilon^2}$$
$$\ln(0.005) \geq -2n\epsilon^2$$
$$- \frac{\ln(0.005)}{2} \leq n\epsilon^2$$
$$2.649 \leq n\epsilon^2$$
$$n \geq \frac{2.649}{\epsilon^2}$$

Say the error bound is $\pm 1%$, i.e., $\epsilon = 0.01$,
$$n \geq \frac{2.649}{0.01^2}$$
$$n \geq 26,491.5$$

Therefore, we can say that for an error bound of 1%, n should be at least 26,492, i.e., at least 26,492 people should be sampled from a million.